In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

cc_apps = pd.read_csv("datasets/cc_approvals.data",header=None)

In [ ]:

cc_apps.head(5)
cc_apps_description = cc_apps.describe()
cc_apps_info = cc_apps.info()
cc_apps = cc_apps.replace('?',np.nan)

In [ ]:
mean = cc_apps.mean()
dicto = {2:mean,7:mean,10:mean,14:mean}
cc_apps.fillna(dicto,inplace=True)
cc_apps.isna().sum()

In [ ]:
for col in cc_apps.columns:
    if cc_apps[col].dtypes == 'object':
        mode = cc_apps[col].value_counts()[0]
        cc_apps.fillna({col:mode},inplace=True)

In [ ]:
le = LabelEncoder()
for col in cc_apps:
    if cc_apps[col].dtypes=='object':
        cc_apps[col]=cc_apps[col].apply(str)
        cc_apps[col]=le.fit_transform(cc_apps[col])


In [ ]:
cc_apps = cc_apps.drop([11,13], axis=1)
cc_apps = cc_apps.values

#Separating features and variables
X,y = cc_apps[:,0:12] , cc_apps[:,13]

# Splitting into test and train datasets
X_train,X_test,y_train,y_test = train_test_split(X,
                                y,
                                test_size=0.33,
                                random_state=42)
#Min-max scaler gives transforms your numerical data into 0-1 
scaler = MinMaxScaler(feature_range=(0,1))
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.fit_transform(X_test)

In [ ]:
logreg = LogisticRegression()
logreg.fit(rescaledX_train,y_train)
y_pred = logreg.predict(rescaledX_test)

# Get the accuracy score of logreg model and print it
print("Accuracy of logistic regression classifier: ", logreg.score(rescaledX_test,
                                                                  y_test))

# Print the confusion matrix of the logreg model
cnf = confusion_matrix(y_test,y_pred)

In [ ]:
tol = [0.01,0.001,0.0001]
max_iter = [100,150,200]
param_grid = dict(tol=tol,max_iter=max_iter)
grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)
rescaledX = scaler.fit_transform(X)
grid_model_result = grid_model.fit(rescaledX,y)

# Summarize results
best_score, best_params = grid_model_result.best_score_, grid_model_result.best_params_
print("Best: %f using %s" % (best_score, best_params))